In [1]:
import torch
import torchvision.models as models
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import DataParallel
from torch.utils.data.sampler import RandomSampler, SequentialSampler
import torch.backends.cudnn as cudnn
import numpy as np

In [2]:
import cv2
import pandas as pd
from copy import deepcopy
from tqdm import tqdm
from importlib import import_module

import os
opj = os.path.join
ope = os.path.exists

In [3]:
from bestfitting.protein_clean.src.net import _init_paths
from bestfitting.protein_clean.src.net import densenet
from bestfitting.protein_clean.src.config.config import *
from bestfitting.protein_clean.src.dataset import protein_dataset
from bestfitting.protein_clean.src import train_net_base
from bestfitting.protein_clean.src import train_cls_net
from bestfitting.protein_clean.src.utils.augment_util import *
from bestfitting.protein_clean.src.net.loss_funcs.kaggle_metric import prob_to_result
from net.tool import *

run on ip-172-31-62-205
run on ip-172-31-62-205


In [99]:
header = 'external'
fname = opj(DATA_DIR, 'meta/%s_meta.csv'%(header))
df = pd.read_csv(fname)
print(fname,':', df.values.shape)

/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/meta/external_meta.csv : (74606, 33)


In [100]:
header = 'train'
fname = opj(DATA_DIR, 'meta/%s_meta.csv'%(header))
df = pd.read_csv(fname)
print(fname,':', df.values.shape)

/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/meta/train_meta.csv : (31072, 31)


In [101]:
fname = opj(DATA_DIR, 'split/train_31072.csv')
df = pd.read_csv(fname)
print(fname,':', df.values.shape)

/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/train_31072.csv : (31072, 31)


In [63]:
fname = opj(DATA_DIR, 'split/test_11702.csv')
df = pd.read_csv(fname)
print(fname,':', df.values.shape)

/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/test_11702.csv : (11702, 2)


In [53]:
split_alias = 'random_folds5'
num = []
for fold in np.arange(5):
    fname = opj(DATA_DIR, 'split', split_alias, "random_train_cv{}.csv".format(fold))
    df = pd.read_csv(fname)
    print(fname,':', df.values.shape)
    num.append(df.values.shape[0])
    
for fold in np.arange(5):
    fname = opj(DATA_DIR, 'split', split_alias, "random_valid_cv{}.csv".format(fold))
    df = pd.read_csv(fname)
    print(fname,':', df.values.shape, ';sum:', num[fold] + df.values.shape[0])    

/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_folds5/random_train_cv0.csv : (24922, 31)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_folds5/random_train_cv1.csv : (24801, 31)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_folds5/random_train_cv2.csv : (24822, 31)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_folds5/random_train_cv3.csv : (24893, 31)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_folds5/random_train_cv4.csv : (24850, 31)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_folds5/random_valid_cv0.csv : (6150, 31) ;sum: 31072
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_folds5/random_valid_cv1.csv : (6271, 31) ;sum: 31072
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_folds5/random_valid_cv2.csv : (6250, 31) ;sum: 31072
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/d

In [54]:
split_alias = 'random_ext_folds5'
num = []
for fold in np.arange(5):
    fname = opj(DATA_DIR, 'split', split_alias, "random_train_cv{}.csv".format(fold))
    df = pd.read_csv(fname)
    print(fname,':', df.values.shape)
    num.append(df.values.shape[0])
    
for fold in np.arange(5):
    fname = opj(DATA_DIR, 'split', split_alias, "random_valid_cv{}.csv".format(fold))
    df = pd.read_csv(fname)
    print(fname,':', df.values.shape, ';sum:', num[fold] + df.values.shape[0])    

/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_folds5/random_train_cv0.csv : (84587, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_folds5/random_train_cv1.csv : (84563, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_folds5/random_train_cv2.csv : (84485, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_folds5/random_train_cv3.csv : (84588, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_folds5/random_train_cv4.csv : (84489, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_folds5/random_valid_cv0.csv : (21091, 33) ;sum: 105678
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_folds5/random_valid_cv1.csv : (21115, 33) ;sum: 105678
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_folds5/random_valid_cv2.csv : (21193, 33) ;sum: 105678
/home/ubuntu/HPA/

In [55]:
split_alias = 'random_ext_noleak_clean_folds5'
num = []
for fold in np.arange(5):
    fname = opj(DATA_DIR, 'split', split_alias, "random_train_cv{}.csv".format(fold))
    df = pd.read_csv(fname)
    print(fname,':', df.values.shape)
    num.append(df.values.shape[0])
    
for fold in np.arange(5):
    fname = opj(DATA_DIR, 'split', split_alias, "random_valid_cv{}.csv".format(fold))
    df = pd.read_csv(fname)
    print(fname,':', df.values.shape, ';sum:', num[fold] + df.values.shape[0])    

/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_noleak_clean_folds5/random_train_cv0.csv : (82749, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_noleak_clean_folds5/random_train_cv1.csv : (82711, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_noleak_clean_folds5/random_train_cv2.csv : (82645, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_noleak_clean_folds5/random_train_cv3.csv : (82767, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_noleak_clean_folds5/random_train_cv4.csv : (82660, 33)
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_noleak_clean_folds5/random_valid_cv0.csv : (19984, 33) ;sum: 102733
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/split/random_ext_noleak_clean_folds5/random_valid_cv1.csv : (20062, 33) ;sum: 102773
/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/dat

In [106]:
img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/train/images_1536'
external_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/train/external_v18_1536'
test_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/test/images_1536'

In [107]:
print(np.sort(os.listdir(img_dir)).shape[0]/4)
print(np.sort(os.listdir(external_img_dir)).shape[0]/4)
print(np.sort(os.listdir(test_img_dir)).shape[0]/4)

31072.0
74596.0
11702.0


In [109]:
img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/tif/train/tifs'
external_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/externel/HPAv18/jpg'
test_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/tif/test/tifs'

In [110]:
print(np.sort(os.listdir(img_dir)).shape[0]/4)
print(np.sort(os.listdir(external_img_dir)).shape[0]/4)
print(np.sort(os.listdir(test_img_dir)).shape[0]/4)


31072.0
74606.0
11702.0


In [167]:
external_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/externel/HPAv18/jpg'
file_list = np.sort(os.listdir(external_img_dir))

header = 'external'
fname = opj(DATA_DIR, 'meta/%s_meta.csv'%(header))
df = pd.read_csv(fname)
external_id = df['Id'].values
external_id = np.sort(external_id)

In [190]:
external_list = external_id.copy()
color_list = ['_blue.jpg', '_green.jpg', '_red.jpg', '_yellow.jpg']
img_id = None
n_iter = 0

for i in range(len(file_list)):
    idx = i%4
    char = color_list[idx]
    img_id0 = img_id
    img_id = file_list[i].replace(char, '')

    if img_id != img_id0:
        external_list[n_iter] = img_id
        n_iter += 1

external_list = np.sort(external_list)

In [192]:
np.sum(external_list == external_id) / len(external_id)

1.0

In [237]:
external_img_dir = '/home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/data/train/external_v18_1536'
file_list = np.sort(os.listdir(external_img_dir))
len(file_list)/4

74596.0

In [251]:
file_list[8720:8730]

array(['12444_610_C8_2_blue.jpg', '12444_610_C8_2_green.jpg',
       '12444_610_C8_2_red.jpg', '12444_610_C8_3_blue.jpg',
       '12444_610_C8_3_green.jpg', '12444_610_C8_3_yellow.jpg',
       '12444_613_C8_1_blue.jpg', '12444_613_C8_1_green.jpg',
       '12444_613_C8_1_red.jpg', '12444_613_C8_1_yellow.jpg'],
      dtype='<U44')

In [252]:
# for i in range(len(file_list)):
#     idx = i%4
#     char = color_list[idx]
#     if file_list[i].find(char) == -1:
#         print('nope', i)

In [241]:
char

'_blue.jpg'

In [242]:
file_list[0].find(char)

14

In [246]:
file_list[0].find('_green.jpg')

-1

In [228]:
external_resize_list = external_id.copy()[:74596]
color_list = ['_blue.jpg', '_green.jpg', '_red.jpg', '_yellow.jpg']
img_id = None
n_iter = 0

for i in range(len(file_list)):
    idx = i%4
    char = color_list[idx]
    img_id0 = img_id
    img_id = file_list[i].replace(char, '')

    if img_id != img_id0:
        external_resize_list[n_iter] = img_id
        n_iter += 1

external_resize_list = np.sort(external_resize_list)

IndexError: index 74596 is out of bounds for axis 0 with size 74596

In [235]:
file_list[50:100]

array(['10019_200_E1_1_red.jpg', '10019_200_E1_1_yellow.jpg',
       '10019_200_E1_2_blue.jpg', '10019_200_E1_2_green.jpg',
       '10019_200_E1_2_red.jpg', '10019_200_E1_2_yellow.jpg',
       '10019_201_E1_1_blue.jpg', '10019_201_E1_1_green.jpg',
       '10019_201_E1_1_red.jpg', '10019_201_E1_1_yellow.jpg',
       '10019_201_E1_2_blue.jpg', '10019_201_E1_2_green.jpg',
       '10019_201_E1_2_red.jpg', '10019_201_E1_2_yellow.jpg',
       '10019_202_E1_1_blue.jpg', '10019_202_E1_1_green.jpg',
       '10019_202_E1_1_red.jpg', '10019_202_E1_1_yellow.jpg',
       '10019_202_E1_2_blue.jpg', '10019_202_E1_2_green.jpg',
       '10019_202_E1_2_red.jpg', '10019_202_E1_2_yellow.jpg',
       '10023_100_C10_1_blue.jpg', '10023_100_C10_1_green.jpg',
       '10023_100_C10_1_red.jpg', '10023_100_C10_1_yellow.jpg',
       '10023_100_C10_2_blue.jpg', '10023_100_C10_2_green.jpg',
       '10023_100_C10_2_red.jpg', '10023_100_C10_2_yellow.jpg',
       '10023_101_C10_1_blue.jpg', '10023_101_C10_1_green.jpg'

In [230]:
i

174380

In [233]:
len(file_list)

298384

In [192]:
np.sum(external_list == external_id) / len(external_id)

1.0